In [ ]:
from PIL import Image
import random
import numpy as np
import matplotlib.pyplot as plt
import time
from matplotlib.pyplot import figure, draw, pause
import matplotlib.colors
from IPython.display import clear_output

In [ ]:
def random_forest(width, height, vegetation_ratio=0.5):
    map_matrix = np.zeros((height, width))
    
    water = 0
    land = 1
    vegetation = 2
    
    for i in range(height):
        for j in range(width):
            
            # 2 tiles water padding
            if i < 2 or i > height - 3 or j < 2 or j > width - 3:
                map_matrix[i][j] = water

            # set initial vegetation
            elif random.uniform(0,1) < vegetation_ratio:
                map_matrix[i][j] = vegetation
            
            # set land
            else:
                map_matrix[i][j] = land
                
    return map_matrix

In [ ]:
class ForestFires:
    def __init__(self, map_matrix, init_burning_ratio, init_ratio_burned):
        self.map_matrix = map_matrix
        self.init_burning_ratio = init_burning_ratio
        self.init_ratio_burned = init_ratio_burned
        
        self.neigh_dirs = [[-1,-1], [-1,0], [0,-1], [1,0], [0,0], [0,1], [1,1], [1,-1], [-1,1]]
        
        self.veg_population = self.set_vegetation_population(self.map_matrix)
        
        self.wind_matrix = np.array([[1,1,1],
                                   [1,1,1],
                                   [1,1,1]])       
            
        
    def set_vegetation_population(self, map_matrix):
        veg_population = {}
        
        # iterate over grid
        for y in range(len(map_matrix)):
            for x in range(len(map_matrix)):
                
                # if vegetation
                if map_matrix[y][x] == 2:
                    
                    # initial propability of vegetation burning
                    if random.uniform(0,1) < self.init_burning_ratio:
                        veg_population[(x,y)] = Vegetation(self.init_ratio_burned, 1)
                        
                    else:
                        veg_population[(x,y)] = Vegetation(0, 1)
                    
        return veg_population
                    
    
        
    def update(self):
        current_veg_pop = self.veg_population.copy()

        for (x,y) in current_veg_pop:
            
            neigh_matrix = np.zeros((3,3))
            
            for coords in self.neigh_dirs:
                neigh_x = x + coords[0]
                neigh_y = y + coords[1]
                
                if self.map_matrix[neigh_x, neigh_y] == 2:
                    
                    neigh_matrix[coords[0] + 1][coords[1] + 1] = self.veg_population[(neigh_y, neigh_x)].ratio_burned
                    
            summed_matrix = np.sum(neigh_matrix * self.wind_matrix)

            current_veg_pop[(x,y)].ratio_burned = (summed_matrix if summed_matrix < 1 else 1)
                    
                
        
    def visualize(self):
        current_map = self.map_matrix.copy()
        
        fire_gradations = []
        
        for (x,y) in self.veg_population:
            p_burned = self.veg_population[(x,y)].ratio_burned
            if p_burned > 0:
                # start stage of fire
                if p_burned <= 0.33:
                    current_map[y][x] = 6
                    if 'yellow' not in fire_gradations:
                        fire_gradations.append('yellow')
                    
                # middle stage of fire
                elif p_burned > 0.33 and p_burned <= 0.66:
                    current_map[y][x] = 4
                    if 'orange' not in fire_gradations:
                        fire_gradations.append('orange')
                    
                # last stage of fire
                elif p_burned > 0.66 and p_burned < 1:
                    current_map[y][x] = 5
                    if 'red' not in fire_gradations:
                        fire_gradations.append('red')
                    
                # burned up
                else:
                    current_map[y][x] = 3
                    if 'black' not in fire_gradations:
                        fire_gradations.append('black')
                        
        
                    
        fire_gradations = sorted(fire_gradations)
        colors = ['blue', 'lightgrey', 'green']
        colors += fire_gradations
#         print(colors)
        
    
        
                    
        plt.imshow(current_map, cmap=matplotlib.colors.ListedColormap(colors))
        plt.show()
        clear_output(wait=True)
    
        
class Vegetation:
    def __init__(self, ratio_burned, height):
    
        # burning or neutral
        self.ratio_burned = ratio_burned
        self.height = height

In [ ]:
map_array = random_forest(150,150, vegetation_ratio=0.2)
forest_sim = ForestFires(map_array, init_burning_ratio=0.1, init_ratio_burned=0.00005)
forest_sim.visualize()
iterations = 20

for i in range(iterations):
    forest_sim.update()
    forest_sim.visualize()
